In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import joblib


In [ ]:
# 2. Load data
# Ganti path_file dengan file yang sesuai dari Kaggle
train_data = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")  # Dataset latih
test_data = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")    # Dataset uji

In [ ]:
# 3. Eksplorasi data
print(train_data.head())
print(train_data.info())

In [ ]:
from transformers import set_seed
import ctypes, gc
import torch
import random
import numpy as np

libc = ctypes.CDLL("libc.so.6")
# Seed the same seed to all 
def seed_everything(seed=42):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    set_seed(seed)
    
def clear_memory():
    libc.malloc_trim(0)
    torch.cuda.empty_cache()
    gc.collect()

SEED = 42
seed_everything(SEED)
# Set the GPUs
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import log_loss
import joblib

In [ ]:
train = pd.read_csv(r'/kaggle/input/llm-classification-finetuning/train.csv')
test = pd.read_csv(r'/kaggle/input/llm-classification-finetuning/test.csv')
sample_submission = pd.read_csv(r'/kaggle/input/llm-classification-finetuning/sample_submission.csv')

print('train data shape :', train.shape)
print('test data shape :', test.shape)
print('sample_submission data shape :', sample_submission.shape)

In [ ]:
train.head()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import hstack
from lightgbm import LGBMClassifier
import gc

y = train[['winner_model_a', 'winner_model_b', 'winner_tie']].idxmax(axis=1)
y = y.map({
    'winner_model_a': 0,
    'winner_model_b': 1,
    'winner_tie': 2
})

df_fit = test if len(test) > 3 else train

tfidf_prompt = TfidfVectorizer(
    max_features=500,
    stop_words='english',
    min_df=0.002,
    ngram_range=(1, 3)
)
count_prompt = CountVectorizer(
    max_features=500,
    stop_words='english',
    min_df=0.002,
    ngram_range=(1, 3)
)

tfidf_response = TfidfVectorizer(
    max_features=1000,
    stop_words='english',
    min_df=0.002,
    ngram_range=(1, 3)
)
count_response = CountVectorizer(
    max_features=1000,
    stop_words='english',
    min_df=0.002,
    ngram_range=(1, 3)
)

tfidf_prompt_char = TfidfVectorizer(
    analyzer='char',
    ngram_range=(1, 3),
    max_features=500,
    min_df=0.002
)
count_prompt_char = CountVectorizer(
    analyzer='char',
    ngram_range=(1, 3),
    max_features=500,
    min_df=0.002
)

tfidf_response_char = TfidfVectorizer(
    analyzer='char',
    ngram_range=(1, 3),
    max_features=1000,
    min_df=0.002
)
count_response_char = CountVectorizer(
    analyzer='char',
    ngram_range=(1, 3),
    max_features=1000,
    min_df=0.002
)

tfidf_prompt.fit(df_fit['prompt'])
count_prompt.fit(df_fit['prompt'])

tfidf_response.fit(pd.concat([df_fit['response_a'], df_fit['response_b']]))
count_response.fit(pd.concat([df_fit['response_a'], df_fit['response_b']]))

tfidf_prompt_char.fit(df_fit['prompt'])
count_prompt_char.fit(df_fit['prompt'])

tfidf_response_char.fit(pd.concat([df_fit['response_a'], df_fit['response_b']]))
count_response_char.fit(pd.concat([df_fit['response_a'], df_fit['response_b']]))

def get_features(df):
    X_prompt_tfidf = tfidf_prompt.transform(df['prompt'])
    X_prompt_count = count_prompt.transform(df['prompt'])
    
    X_prompt_tfidf_char = tfidf_prompt_char.transform(df['prompt'])
    X_prompt_count_char = count_prompt_char.transform(df['prompt'])
    
    X_prompt_combined = hstack([
        X_prompt_tfidf, 
        X_prompt_count, 
        X_prompt_tfidf_char, 
        X_prompt_count_char
    ])
    
    response_combined = pd.concat([df['response_a'], df['response_b']], axis=0)
    X_response_tfidf = tfidf_response.transform(response_combined)
    X_response_count = count_response.transform(response_combined)
    
    X_response_tfidf_char = tfidf_response_char.transform(response_combined)
    X_response_count_char = count_response_char.transform(response_combined)
    
    n = len(df)
    X_response_a_tfidf = X_response_tfidf[:n]
    X_response_b_tfidf = X_response_tfidf[n:]
    
    X_response_a_count = X_response_count[:n]
    X_response_b_count = X_response_count[n:]
    
    X_response_a_tfidf_char = X_response_tfidf_char[:n]
    X_response_b_tfidf_char = X_response_tfidf_char[n:]
    
    X_response_a_count_char = X_response_count_char[:n]
    X_response_b_count_char = X_response_count_char[n:]
    
    afeat = hstack([
        X_response_a_tfidf, 
        X_response_a_count,
        X_response_a_tfidf_char,
        X_response_a_count_char
    ])
    bfeat = hstack([
        X_response_b_tfidf, 
        X_response_b_count,
        X_response_b_tfidf_char,
        X_response_b_count_char
    ])
    
    v = hstack([
        afeat - bfeat, 
        abs(afeat - bfeat)
    ])
    
    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)
    
    extras.append(df['prompt'].str.len().values)
    extras.append(df['prompt'].str.split().apply(lambda x: len(x)).values)
    
    extras = np.stack(extras, axis=1)
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])
    
    final_features = hstack([v, extras, X_prompt_combined])
    return final_features.tocsr()

In [ ]:
X_train = get_features(train)
X_test = get_features(test)


# def feature_select_wrapper(X, y, cumulative_importance=0.95, n_splits=5, random_state=42):

#     skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

#     fse = np.zeros(X.shape[1])

#     for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):

#         X_train_fold, X_val_fold = X[train_idx], X[val_idx]
#         y_train_fold, y_val_fold = y[train_idx], y[val_idx]

#         model = LGBMClassifier(
#             objective='multiclass',
#             num_class=3,
#             learning_rate=0.05,
#             n_estimators=1000,
#             random_state=random_state,
#             n_jobs=-1
#         )

#         model.fit(
#             X_train_fold, y_train_fold,
#             eval_set=[(X_val_fold, y_val_fold)],
#             eval_metric='multi_logloss',
#         )

#         importances = model.feature_importances_

#         fse += importances

#     fse /= n_splits

#     sorted_indices = np.argsort(fse)[::-1]
#     sorted_importances = fse[sorted_indices]

#     cumulative_importance_values = np.cumsum(sorted_importances) / np.sum(sorted_importances)


#     cutoff_index = np.searchsorted(cumulative_importance_values, cumulative_importance) + 1
#     feature_select_indices = sorted_indices[:cutoff_index].tolist()


#     return feature_select_indices

# selected_feature_indices = feature_select_wrapper(
#     X=X_train,
#     y=y,
#     cumulative_importance=0.9,
#     n_splits=5,
#     random_state=42
# )

# X_train_selected = X_train[:, selected_feature_indices]
# X_test_selected = X_test[:, selected_feature_indices]

In [ ]:
def cross_validate_and_save_models(X, y, n_splits=10, random_state=42, save_path='models_and_weights.joblib'):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    log_losses = []
    weights = []
    models = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        print(f'Fold {fold}/{n_splits}')

        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        # model = XGBClassifier(
        #     objective='multi:softprob',
        #     eval_metric='mlogloss',
        #     num_class=3,
        #     random_state=random_state,
        #     n_estimators=1000,
        #     learning_rate=0.05
        # )

        # model.fit(
        #     X_train_fold, y_train_fold,
        #     eval_set=[(X_val_fold, y_val_fold)],
        #     early_stopping_rounds=50,
        #     verbose=100
        # )
        
        model = LGBMClassifier(
            objective='multiclass',
            num_class=3,
            learning_rate=0.05,
            n_estimators=1000,
            random_state=random_state,
            n_jobs=-1
        )

        model.fit(
            X_train_fold, y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            eval_metric='multi_logloss',
        )
        y_val_pred = model.predict_proba(X_val_fold)
        loss = log_loss(y_val_fold, y_val_pred)
        log_losses.append(loss)
        print(f'Fold {fold} Log Loss: {loss}')

        weight = 1.0 / loss
        weights.append(weight)

        models.append(model)

    average_log_loss = np.mean(log_losses)
    print(f'Average {n_splits}-Fold Log Loss: {average_log_loss}')

    models_and_weights = {
        'models': models, 
        'weights': weights 
    }

    joblib.dump(models_and_weights, save_path)
    print(f'All models and weights have been saved to {save_path}')

cross_validate_and_save_models(
    X=X_train,
    y=y,
    n_splits=10,
    random_state=42,
    save_path='models_and_weights.joblib'
)

In [ ]:
def load_models_and_weights(save_path='models_and_weights.joblib'):
    models_and_weights = joblib.load(save_path)
    models = models_and_weights['models']
    weights = models_and_weights['weights']
    print(f'Loaded {len(models)} models with corresponding weights.')
    return models, weights

def weighted_predict(models, weights, X_test):
    test_preds = np.zeros((X_test.shape[0], 3))
    for model, weight in zip(models, weights):
        y_pred = model.predict_proba(X_test)
        test_preds += y_pred * weight
    test_preds_weighted = test_preds / np.sum(weights)
    return test_preds_weighted

models, weights = load_models_and_weights(save_path='models_and_weights.joblib')

y_test_proba = weighted_predict(models, weights, X_test)

In [ ]:
submission = pd.DataFrame({
    'id': test['id'],
    'winner_model_a': y_test_proba[:, 0],
    'winner_model_b': y_test_proba[:, 1],
    'winner_tie': y_test_proba[:, 2]
})

In [ ]:
submission.to_csv('submission.csv', index=False)
submission